# Code examples for interacting with Mondo using OAK

### OAK
OAK, the Ontology Access Kit, is a Python library for common ontology operations and provides a variety of adapters to connect to ontologies. More information is available at: https://incatools.github.io/ontology-access-kit/index.html

<a id="overview"></a>
## Notebook Overview

This notebook has 5 sections as listed below.

* **Prepare Working Environment**
    * [Create working environment](#env)

* **Set-up Notebook**
    * [Install Packages](#install)
    * [Load Imports](#imports)

* **Get Ontology Database file**
    * [Download ontology database file](#download)

* **Create Helper functions**
    * [Create functions to access ontology class property values](#functions)
        * [Get Ancestors](#ancestors)
        * [Get Descendants](#descendants)
        * [Get Property Value](#generic)
        * [Get Mappings](#mappings)
    * [Other function examples](#other)

* **Build up the Dataframe**
    * [Get Data about a Class](#data)
        * [Get children of the Class](#children)
        * [Get parents of the classes](#parents)
        * [Get children of the classes](#children_classes)
        * [Get their disease mappings](#disease_mappings)
        * [Get their conformsTo property](#conforms_to)
        * [Get their cross-species analog property](#cross_species_analog)

<a id="env"></a>
### Create a Python virtual environment or new Conda environment
To avoid possible package dependency issues, create a new env to use this notebook. See [Getting Started](https://incatools.github.io/ontology-access-kit/intro/tutorial01.html#part-1-getting-started) with OAK for a reminder on how to create a Python environment or the [Conda Cheat Sheet](https://docs.conda.io/projects/conda/en/4.6.0/_downloads/52a95608c49671267e40c689e0bc00ca/conda-cheatsheet.pdf) for commands to create a new conda environment. The base packages needed are `Python` and `pandas`.

[Back](#overview)

---
<a id="install"></a>
### Install Python packages needed for this notebook

In [1]:
# Install Python packages
# !pip install oaklib

In [2]:
# Test OAK Installation
# !runoak --help

[Back](#overview)

---
<a id="imports"></a>
### Load Imports

In [3]:
# Load Imports
import pandas as pd
from pathlib import Path
from oaklib import get_adapter
from oaklib.mappers import OntologyMetadataMapper
from oaklib.datamodels.vocabulary import IS_A, PART_OF, SEMAPV, OWL_OBJECT_PROPERTY, OWL_ANNOTATION_PROPERTY
from typing import Tuple, List

# Configure dataframe display
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

[Back](#overview)

---
<a id="download"></a>
### Download a SQLite version of your ontology of interest
There are a few options to download a SQLite version of an ontology file to use with oaklib. This notebook will use the `get_adapter()` method, which creates a connection handle to the SQLite database file found at `~/.data/oaklib` and if the ontology database file does not already exist it is downloaded. 

More information on how to download a SQLite ontology file can be found here: https://incatools.github.io/ontology-access-kit/intro/tutorial07.html#download-a-sqlite-file

In [4]:
# Create a wrapper to connect to the mondo database
adapter = get_adapter(f"sqlite:obo:mondo")

### Check the database connection and get the ontology metadata

In [5]:
# Get ontology metadata
for ont in adapter.ontologies():
    ontology_metadata = adapter.ontology_metadata_map(ont)

# Display ontology metadata
display(ontology_metadata)

{'id': ['obo:mondo.owl'],
 'IAO:0000700': ['MONDO:0042489',
  'MONDO:0021178',
  'MONDO:0021125',
  'MONDO:0000001'],
 'dce:description': ['A semi-automatically constructed ontology that merges in multiple disease resources to yield a coherent merged ontology.'],
 'dce:title': ['Mondo Disease Ontology'],
 'dcterms:license': ['<http://creativecommons.org/licenses/by/4.0/>'],
 'dcterms:source': ['<https://rarediseases.info.nih.gov/>',
  '<https://id.nlm.nih.gov/mesh/>',
  '<http://www.orpha.net/ontology/orphanet.owl>',
  'obo:uberon.owl',
  'obo:ncit.owl',
  'obo:ncbitaxon.owl',
  'obo:mf.owl',
  'obo:hp.owl',
  'obo:go.owl',
  'obo:envo.owl',
  'obo:doid.owl',
  'obo:chebi.owl'],
 'foaf:homepage': ['http://obofoundry.org/ontology/mondo.html'],
 'oio:hasOBOFormatVersion': ['1.2'],
 'owl:versionIRI': ['obo:mondo/releases/2023-09-12/mondo.owl'],
 'rdf:type': ['owl:Ontology'],
 'rdfs:comment': ['Includes Ontology(OntologyID(Anonymous-11)) [Axioms: 74703 Logical Axioms: 0]'],
 'sh:prefix': [

[Back](#overview)

---
<a id="functions"></a>
### Create functions to get annotations for each class

<a id="ancestors"></a>
#### Get Ancestors

In [6]:
# Create function to get all ancestors
def _get_ancestors(curie: str) -> List[Tuple[str, str]]:
    all_ancestors_data = [(adapter.label(ancestor), ancestor) 
                     for ancestor in adapter.ancestors(curie, predicates=[IS_A]) 
                     if ancestor != curie]
  
    
    return all_ancestors_data

In [7]:
# Test _get_ancestors()
_get_ancestors('MONDO:0018076')

[('entity', 'BFO:0000001'),
 ('continuant', 'BFO:0000002'),
 ('disposition', 'BFO:0000016'),
 ('realizable entity', 'BFO:0000017'),
 ('specifically dependent continuant', 'BFO:0000020'),
 ('disease', 'MONDO:0000001'),
 ('primary bacterial infectious disease', 'MONDO:0000314'),
 ('bacterial infectious disease', 'MONDO:0005113'),
 ('infectious disease', 'MONDO:0005550'),
 ('mycobacterial infectious disease', 'MONDO:0020590'),
 ('human disease', 'MONDO:0700096'),
 ('disease', 'OGMS:0000031')]

[Back](#overview)

---
<a id="descendants"></a>
#### Get Descendants

In [8]:
# Create function to get all descendants
# NOTE: The input curie is returned from adapter.descendants, but is filtered out from _get_descendants()
def _get_descendants(curie: str) -> List[Tuple[str, str]]:
    all_descendants_data = [(adapter.label(child), child) 
                            for child in adapter.descendants([curie], predicates=[IS_A])
                           if child != curie]
    
    # Sort the results
    all_descendants_data.sort(key=lambda x: x[0])
    
    return all_descendants_data

In [9]:
# Test _get_descendants()
_get_descendants('MONDO:0018076')[:3]

[('abdominal tuberculosis', 'MONDO:0000369'),
 ('active tuberculosis', 'MONDO:0100481'),
 ('cardiac tuberculosis', 'MONDO:0000813')]

[Back](#overview)

---
<a id="generic"></a>
#### Generic method to get any property value

In [10]:
# Generic method to get value from class entity metadata map
def _get_entity_metadata_map_value(curie: str, property_value: str) -> List[str]:
    return adapter.entity_metadata_map(curie).get(property_value)

In [11]:
# Test _get_entity_metadata_map_value
display(_get_entity_metadata_map_value('MONDO:0024945', '<https://w3id.org/semapv/vocab/crossSpeciesExactMatch>'))

display(_get_entity_metadata_map_value('MONDO:0000827', 'dcterms:conformsTo'))

['MONDO:0002251']

['obo:mondo/patterns/infectious_disease_by_agent.yaml']

[Back](#overview)

---
<a id="mappings"></a>
#### Get mappings

In [12]:
# Get disease mappings
def _get_mappings(curie: str, predicate_id='skos:exactMatch') -> List[str]:
    return [mapping.object_id for mapping in adapter.sssom_mappings([curie]) if predicate_id == mapping.predicate_id]

In [13]:
# Test - get mappings
_get_mappings('MONDO:0018076')

['ORDO:3389',
 'NCIT:C3423',
 'DOID:399',
 'ICD10CM:A15-A19',
 '<http://linkedlifedata.com/resource/umls/id/C0151332>',
 '<http://linkedlifedata.com/resource/umls/id/C0041296>',
 '<http://identifiers.org/snomedct/56717001>',
 '<http://identifiers.org/mesh/D014376>']

[Back](#overview)

---
<a id="other"></a>
### Other examples

In [14]:
# Get all Annotation properties
def _get_annotation_properties():
    all_anno_properties = []
    for entity in adapter.entities(filter_obsoletes=True, owl_type = "owl:AnnotationProperty"):
        display(entity)

In [15]:
# Test - Get all Annotation properties
# _get_annotation_properties()

---

In [16]:
# Get all Object properties
def _get_object_properties():
    all_obj_properties = []
    for entity in adapter.entities(filter_obsoletes=True, owl_type = "owl:ObjectProperty"):
        display(entity)

In [17]:
# Test - Get all Object Properties
# _get_object_properties()

---

In [18]:
# Get all relationships for a class
def _get_all_relationships(curie: str) -> str:
    for s, p, o in sorted(adapter.relationships(subjects=[curie])):
        print(f"{p} {o} '{adapter.label(o)}'")

In [19]:
# Test - Get all relationships for a class
# _get_all_relationships('MONDO:0018076')

[Back](#overview)

---
<a id='data'></a>
### ** Get data about a Class
The information collected for the Class of interest will include:
- curie
- label
- list of ancestors
- list of descendants
- cross-species analog relation
- conformsTo relation
- disease mappings

In [20]:
# Let's explore the ontology and get some initial data by getting all children of a class 

# Get all descendants for a CURIE
curie = "MONDO:0018076"

#NOTE: The result of adapter.descendants() includes the input curie so the count is all children plus one.
all_descendants = set(adapter.descendants([curie], predicates=[IS_A]))


display(len(all_descendants), all_descendants)

53

{'MONDO:0000271',
 'MONDO:0000368',
 'MONDO:0000369',
 'MONDO:0000813',
 'MONDO:0001537',
 'MONDO:0001678',
 'MONDO:0002226',
 'MONDO:0004189',
 'MONDO:0004272',
 'MONDO:0004517',
 'MONDO:0005696',
 'MONDO:0005768',
 'MONDO:0005787',
 'MONDO:0005819',
 'MONDO:0005831',
 'MONDO:0005848',
 'MONDO:0005861',
 'MONDO:0005887',
 'MONDO:0005903',
 'MONDO:0005922',
 'MONDO:0005938',
 'MONDO:0005962',
 'MONDO:0005967',
 'MONDO:0005999',
 'MONDO:0006000',
 'MONDO:0006002',
 'MONDO:0006042',
 'MONDO:0006052',
 'MONDO:0006743',
 'MONDO:0006838',
 'MONDO:0006845',
 'MONDO:0006876',
 'MONDO:0018076',
 'MONDO:0021804',
 'MONDO:0021943',
 'MONDO:0021948',
 'MONDO:0021953',
 'MONDO:0023153',
 'MONDO:0040753',
 'MONDO:0041806',
 'MONDO:0043836',
 'MONDO:0100479',
 'MONDO:0100481',
 'MONDO:0100482',
 'MONDO:0100483',
 'MONDO:0850417',
 'MONDO:0957461',
 'MONDO:0957462',
 'MONDO:0957463',
 'MONDO:0957464',
 'MONDO:0957465',
 'MONDO:0957466',
 'MONDO:0957467'}

In [21]:
# View results for CURIE in OLS for comparison
import webbrowser

# convert curie to purl
url = adapter.curie_to_uri(curie)

# open ols browser page for purl
webbrowser.open(url)

True

[Back](#overview)

---
<a id='children'></a>
### Get Children of a Class

In [22]:
# Get all Class data for a Curie of Interest
curie = "MONDO:0018076" # tuberculosis

columns = {'curie': str, 'label': str}
class_df = pd.DataFrame(columns=columns)

# First, get all children and their labels for the curie of interest
class_df[['label', 'curie']] = pd.DataFrame(_get_descendants(curie))

display(class_df)

,curie,label
0,MONDO:0000369,abdominal tuberculosis
1,MONDO:0100481,active tuberculosis
2,MONDO:0000813,cardiac tuberculosis
3,MONDO:0005696,central nervous system tuberculosis
4,MONDO:0021948,cutaneous tuberculosis
5,MONDO:0041806,drug-resistant tuberculosis
6,MONDO:0006743,endocrine tuberculosis
7,MONDO:0004189,esophageal tuberculosis
8,MONDO:0100482,extensively drug-resistant tuberculosis
9,MONDO:0000368,extrapulmonary tuberculosis


[Back](#overview)

---
<a id='parents'></a>
### Get Parents of the Classes

In [23]:
# Add all ancestor data to the dataframe
class_df['parents'] = class_df['curie'].apply(lambda x: _get_ancestors(x))
display(class_df)

,curie,label,parents
0,MONDO:0000369,abdominal tuberculosis,"[(entity, BFO:0000001), (continuant, BFO:0000002), (disposition, BFO:0000016), (realizable entity, BFO:0000017), (specifically dependent continuant, BFO:0000020), (disease, MONDO:0000001), (primary bacterial infectious disease, MONDO:0000314), (extrapulmonary tuberculosis, MONDO:0000368), (bacterial infectious disease, MONDO:0005113), (infectious disease, MONDO:0005550), (tuberculosis, MONDO:0018076), (mycobacterial infectious disease, MONDO:0020590), (human disease, MONDO:0700096), (disease, OGMS:0000031)]"
1,MONDO:0100481,active tuberculosis,"[(entity, BFO:0000001), (continuant, BFO:0000002), (disposition, BFO:0000016), (realizable entity, BFO:0000017), (specifically dependent continuant, BFO:0000020), (disease, MONDO:0000001), (primary bacterial infectious disease, MONDO:0000314), (bacterial infectious disease, MONDO:0005113), (infectious disease, MONDO:0005550), (tuberculosis, MONDO:0018076), (mycobacterial infectious disease, MONDO:0020590), (human disease, MONDO:0700096), (disease, OGMS:0000031)]"
2,MONDO:0000813,cardiac tuberculosis,"[(entity, BFO:0000001), (continuant, BFO:0000002), (disposition, BFO:0000016), (realizable entity, BFO:0000017), (specifically dependent continuant, BFO:0000020), (disease, MONDO:0000001), (primary bacterial infectious disease, MONDO:0000314), (extrapulmonary tuberculosis, MONDO:0000368), (cardiovascular disorder, MONDO:0004995), (bacterial infectious disease, MONDO:0005113), (heart disorder, MONDO:0005267), (infectious disease, MONDO:0005550), (tuberculosis, MONDO:0018076), (mycobacterial infectious disease, MONDO:0020590), (human disease, MONDO:0700096), (disease, OGMS:0000031)]"
3,MONDO:0005696,central nervous system tuberculosis,"[(entity, BFO:0000001), (continuant, BFO:0000002), (disposition, BFO:0000016), (realizable entity, BFO:0000017), (specifically dependent continuant, BFO:0000020), (disease, MONDO:0000001), (primary bacterial infectious disease, MONDO:0000314), (extrapulmonary tuberculosis, MONDO:0000368), (central nervous system disorder, MONDO:0002602), (nervous system disorder, MONDO:0005071), (bacterial infectious disease, MONDO:0005113), (infectious disease, MONDO:0005550), (tuberculosis, MONDO:0018076), (infectious disorder of the nervous system, MONDO:0020010), (mycobacterial infectious disease, MONDO:0020590), (central nervous system infectious disorder, MONDO:0024619), (human disease, MONDO:0700096), (disease, OGMS:0000031)]"
4,MONDO:0021948,cutaneous tuberculosis,"[(entity, BFO:0000001), (continuant, BFO:0000002), (disposition, BFO:0000016), (realizable entity, BFO:0000017), (specifically dependent continuant, BFO:0000020), (disease, MONDO:0000001), (primary bacterial infectious disease, MONDO:0000314), (extrapulmonary tuberculosis, MONDO:0000368), (integumentary system disorder, MONDO:0002051), (skin disorder, MONDO:0005093), (bacterial infectious disease, MONDO:0005113), (infectious disease, MONDO:0005550), (tuberculosis, MONDO:0018076), (mycobacterial infectious disease, MONDO:0020590), (skin disorder caused by infection, MONDO:0024294), (skin disease caused by bacterial infection, MONDO:0024295), (human disease, MONDO:0700096), (disease, OGMS:0000031)]"
5,MONDO:0041806,drug-resistant tuberculosis,"[(entity, BFO:0000001), (continuant, BFO:0000002), (disposition, BFO:0000016), (realizable entity, BFO:0000017), (specifically dependent continuant, BFO:0000020), (disease, MONDO:0000001), (primary bacterial infectious disease, MONDO:0000314), (bacterial infectious disease, MONDO:0005113), (infectious disease, MONDO:0005550), (tuberculosis, MONDO:0018076), (mycobacterial infectious disease, MONDO:0020590), (human disease, MONDO:0700096), (disease, OGMS:0000031)]"
6,MONDO:0006743,endocrine tuberculosis,"[(entity, BFO:0000001), (continuant, BFO:0000002), (disposition, BFO:0000016), (realizable entity, BFO:0000017), (specifically dependent continuant, BFO:0000020), (disease, MONDO:0000001), (primary bacterial infec

[Back](#overview)

---
<a id='children_classes'></a>
### Get Children of the Classes

In [24]:
# Add all children to the dataframe
class_df['children'] = class_df['curie'].apply(lambda x: _get_descendants(x))
display(class_df)

,curie,label,parents,children
0,MONDO:0000369,abdominal tuberculosis,"[(entity, BFO:0000001), (continuant, BFO:0000002), (disposition, BFO:0000016), (realizable entity, BFO:0000017), (specifically dependent continuant, BFO:0000020), (disease, MONDO:0000001), (primary bacterial infectious disease, MONDO:0000314), (extrapulmonary tuberculosis, MONDO:0000368), (bacterial infectious disease, MONDO:0005113), (infectious disease, MONDO:0005550), (tuberculosis, MONDO:0018076), (mycobacterial infectious disease, MONDO:0020590), (human disease, MONDO:0700096), (disease, OGMS:0000031)]","[(hepatic tuberculosis, MONDO:0005787), (renal tuberculosis, MONDO:0005938), (splenic tuberculosis, MONDO:0005967), (tuberculous ascites, MONDO:0023153), (tuberculous peritonitis, MONDO:0006000)]"
1,MONDO:0100481,active tuberculosis,"[(entity, BFO:0000001), (continuant, BFO:0000002), (disposition, BFO:0000016), (realizable entity, BFO:0000017), (specifically dependent continuant, BFO:0000020), (disease, MONDO:0000001), (primary bacterial infectious disease, MONDO:0000314), (bacterial infectious disease, MONDO:0005113), (infectious disease, MONDO:0005550), (tuberculosis, MONDO:0018076), (mycobacterial infectious disease, MONDO:0020590), (human disease, MONDO:0700096), (disease, OGMS:0000031)]",[]
2,MONDO:0000813,cardiac tuberculosis,"[(entity, BFO:0000001), (continuant, BFO:0000002), (disposition, BFO:0000016), (realizable entity, BFO:0000017), (specifically dependent continuant, BFO:0000020), (disease, MONDO:0000001), (primary bacterial infectious disease, MONDO:0000314), (extrapulmonary tuberculosis, MONDO:0000368), (cardiovascular disorder, MONDO:0004995), (bacterial infectious disease, MONDO:0005113), (heart disorder, MONDO:0005267), (infectious disease, MONDO:0005550), (tuberculosis, MONDO:0018076), (mycobacterial infectious disease, MONDO:0020590), (human disease, MONDO:0700096), (disease, OGMS:0000031)]",[]
3,MONDO:0005696,central nervous system tuberculosis,"[(entity, BFO:0000001), (continuant, BFO:0000002), (disposition, BFO:0000016), (realizable entity, BFO:0000017), (specifically dependent continuant, BFO:0000020), (disease, MONDO:0000001), (primary bacterial infectious disease, MONDO:0000314), (extrapulmonary tuberculosis, MONDO:0000368), (central nervous system disorder, MONDO:0002602), (nervous system disorder, MONDO:0005071), (bacterial infectious disease, MONDO:0005113), (infectious disease, MONDO:0005550), (tuberculosis, MONDO:0018076), (infectious disorder of the nervous system, MONDO:0020010), (mycobacterial infectious disease, MONDO:0020590), (central nervous system infectious disorder, MONDO:0024619), (human disease, MONDO:0700096), (disease, OGMS:0000031)]","[(meningeal tuberculosis, MONDO:0006042)]"
4,MONDO:0021948,cutaneous tuberculosis,"[(entity, BFO:0000001), (continuant, BFO:0000002), (disposition, BFO:0000016), (realizable entity, BFO:0000017), (specifically dependent continuant, BFO:0000020), (disease, MONDO:0000001), (primary bacterial infectious disease, MONDO:0000314), (extrapulmonary tuberculosis, MONDO:0000368), (integumentary system disorder, MONDO:0002051), (skin disorder, MONDO:0005093), (bacterial infectious disease, MONDO:0005113), (infectious disease, MONDO:0005550), (tuberculosis, MONDO:0018076), (mycobacterial infectious disease, MONDO:0020590), (skin disorder caused by infection, MONDO:0024294), (skin disease caused by bacterial infection, MONDO:0024295), (human disease, MONDO:0700096), (disease, OGMS:0000031)]","[(primary cutaneous tuberculosis, MONDO:0957464)]"
5,MONDO:0041806,drug-resistant tuberculosis,"[(entity, BFO:0000001), (continuant, BFO:0000002), (disposition, BFO:0000016), (realizable entity, BFO:0000017), (specifically dependent continuant, BFO:0000020), (disease, MONDO:0000001), (primary bacterial infectious disease, MONDO:0000314), (bacterial infectious disease, MONDO:0005113), (infectious disease, MONDO:0005550), (tuberculosis, MONDO:0018076), (mycobacterial infectious disease, MONDO:0020590), (h

[Back](#overview)

---
<a id='disease_mappings'></a>
### Get Disease mappings of the Classes

In [25]:
# Add mappings to the dataframe
match_type = 'skos:exactMatch'

class_df['mappings'] = class_df['curie'].apply(lambda x: _get_mappings(x, match_type))
display(class_df)

,curie,label,parents,children,mappings
0,MONDO:0000369,abdominal tuberculosis,"[(entity, BFO:0000001), (continuant, BFO:0000002), (disposition, BFO:0000016), (realizable entity, BFO:0000017), (specifically dependent continuant, BFO:0000020), (disease, MONDO:0000001), (primary bacterial infectious disease, MONDO:0000314), (extrapulmonary tuberculosis, MONDO:0000368), (bacterial infectious disease, MONDO:0005113), (infectious disease, MONDO:0005550), (tuberculosis, MONDO:0018076), (mycobacterial infectious disease, MONDO:0020590), (human disease, MONDO:0700096), (disease, OGMS:0000031)]","[(hepatic tuberculosis, MONDO:0005787), (renal tuberculosis, MONDO:0005938), (splenic tuberculosis, MONDO:0005967), (tuberculous ascites, MONDO:0023153), (tuberculous peritonitis, MONDO:0006000)]","[DOID:0050599, <http://linkedlifedata.com/resource/umls/id/C0740652>, <http://identifiers.org/snomedct/447330002>]"
1,MONDO:0100481,active tuberculosis,"[(entity, BFO:0000001), (continuant, BFO:0000002), (disposition, BFO:0000016), (realizable entity, BFO:0000017), (specifically dependent continuant, BFO:0000020), (disease, MONDO:0000001), (primary bacterial infectious disease, MONDO:0000314), (bacterial infectious disease, MONDO:0005113), (infectious disease, MONDO:0005550), (tuberculosis, MONDO:0018076), (mycobacterial infectious disease, MONDO:0020590), (human disease, MONDO:0700096), (disease, OGMS:0000031)]",[],[]
2,MONDO:0000813,cardiac tuberculosis,"[(entity, BFO:0000001), (continuant, BFO:0000002), (disposition, BFO:0000016), (realizable entity, BFO:0000017), (specifically dependent continuant, BFO:0000020), (disease, MONDO:0000001), (primary bacterial infectious disease, MONDO:0000314), (extrapulmonary tuberculosis, MONDO:0000368), (cardiovascular disorder, MONDO:0004995), (bacterial infectious disease, MONDO:0005113), (heart disorder, MONDO:0005267), (infectious disease, MONDO:0005550), (tuberculosis, MONDO:0018076), (mycobacterial infectious disease, MONDO:0020590), (human disease, MONDO:0700096), (disease, OGMS:0000031)]",[],"[DOID:0060570, <http://linkedlifedata.com/resource/umls/id/C0041308>, <http://identifiers.org/mesh/D014381>]"
3,MONDO:0005696,central nervous system tuberculosis,"[(entity, BFO:0000001), (continuant, BFO:0000002), (disposition, BFO:0000016), (realizable entity, BFO:0000017), (specifically dependent continuant, BFO:0000020), (disease, MONDO:0000001), (primary bacterial infectious disease, MONDO:0000314), (extrapulmonary tuberculosis, MONDO:0000368), (central nervous system disorder, MONDO:0002602), (nervous system disorder, MONDO:0005071), (bacterial infectious disease, MONDO:0005113), (infectious disease, MONDO:0005550), (tuberculosis, MONDO:0018076), (infectious disorder of the nervous system, MONDO:0020010), (mycobacterial infectious disease, MONDO:0020590), (central nervous system infectious disorder, MONDO:0024619), (human disease, MONDO:0700096), (disease, OGMS:0000031)]","[(meningeal tuberculosis, MONDO:0006042)]","[ORDO:641396, DOID:1638, <http://linkedlifedata.com/resource/umls/id/C0275904>, <http://identifiers.org/snomedct/186217006>]"
4,MONDO:0021948,cutaneous tuberculosis,"[(entity, BFO:0000001), (continuant, BFO:0000002), (disposition, BFO:0000016), (realizable entity, BFO:0000017), (specifically dependent continuant, BFO:0000020), (disease, MONDO:0000001), (primary bacterial infectious disease, MONDO:0000314), (extrapulmonary tuberculosis, MONDO:0000368), (integumentary system disorder, MONDO:0002051), (skin disorder, MONDO:0005093), (bacterial infectious disease, MONDO:0005113), (infectious disease, MONDO:0005550), (tuberculosis, MONDO:0018076), (mycobacterial infectious disease, MONDO:0020590), (skin disorder caused by infection, MONDO:0024294), (skin disease caused by bacterial infection, MONDO:0024295), (human disease, MONDO:0700096), (disease, OGMS:0000031)]","[(primary cutaneous tuberculosis, MONDO:0957464)]","[<http://linkedlifedata.com/resource/umls/id/C0041309>, <http://identifiers.org/snomedct/

[Back](#overview)

---
<a id='conforms_to'></a>
### Get conformsTo

In [26]:
# Get values for conformsTo
conforms_to = 'dcterms:conformsTo'

class_df['conforms_to'] = class_df['curie'].apply(lambda x: _get_entity_metadata_map_value(x, conforms_to))
display(class_df)

,curie,label,parents,children,mappings,conforms_to
0,MONDO:0000369,abdominal tuberculosis,"[(entity, BFO:0000001), (continuant, BFO:0000002), (disposition, BFO:0000016), (realizable entity, BFO:0000017), (specifically dependent continuant, BFO:0000020), (disease, MONDO:0000001), (primary bacterial infectious disease, MONDO:0000314), (extrapulmonary tuberculosis, MONDO:0000368), (bacterial infectious disease, MONDO:0005113), (infectious disease, MONDO:0005550), (tuberculosis, MONDO:0018076), (mycobacterial infectious disease, MONDO:0020590), (human disease, MONDO:0700096), (disease, OGMS:0000031)]","[(hepatic tuberculosis, MONDO:0005787), (renal tuberculosis, MONDO:0005938), (splenic tuberculosis, MONDO:0005967), (tuberculous ascites, MONDO:0023153), (tuberculous peritonitis, MONDO:0006000)]","[DOID:0050599, <http://linkedlifedata.com/resource/umls/id/C0740652>, <http://identifiers.org/snomedct/447330002>]",[obo:mondo/patterns/location.yaml]
1,MONDO:0100481,active tuberculosis,"[(entity, BFO:0000001), (continuant, BFO:0000002), (disposition, BFO:0000016), (realizable entity, BFO:0000017), (specifically dependent continuant, BFO:0000020), (disease, MONDO:0000001), (primary bacterial infectious disease, MONDO:0000314), (bacterial infectious disease, MONDO:0005113), (infectious disease, MONDO:0005550), (tuberculosis, MONDO:0018076), (mycobacterial infectious disease, MONDO:0020590), (human disease, MONDO:0700096), (disease, OGMS:0000031)]",[],[],None
2,MONDO:0000813,cardiac tuberculosis,"[(entity, BFO:0000001), (continuant, BFO:0000002), (disposition, BFO:0000016), (realizable entity, BFO:0000017), (specifically dependent continuant, BFO:0000020), (disease, MONDO:0000001), (primary bacterial infectious disease, MONDO:0000314), (extrapulmonary tuberculosis, MONDO:0000368), (cardiovascular disorder, MONDO:0004995), (bacterial infectious disease, MONDO:0005113), (heart disorder, MONDO:0005267), (infectious disease, MONDO:0005550), (tuberculosis, MONDO:0018076), (mycobacterial infectious disease, MONDO:0020590), (human disease, MONDO:0700096), (disease, OGMS:0000031)]",[],"[DOID:0060570, <http://linkedlifedata.com/resource/umls/id/C0041308>, <http://identifiers.org/mesh/D014381>]",[obo:mondo/patterns/location.yaml]
3,MONDO:0005696,central nervous system tuberculosis,"[(entity, BFO:0000001), (continuant, BFO:0000002), (disposition, BFO:0000016), (realizable entity, BFO:0000017), (specifically dependent continuant, BFO:0000020), (disease, MONDO:0000001), (primary bacterial infectious disease, MONDO:0000314), (extrapulmonary tuberculosis, MONDO:0000368), (central nervous system disorder, MONDO:0002602), (nervous system disorder, MONDO:0005071), (bacterial infectious disease, MONDO:0005113), (infectious disease, MONDO:0005550), (tuberculosis, MONDO:0018076), (infectious disorder of the nervous system, MONDO:0020010), (mycobacterial infectious disease, MONDO:0020590), (central nervous system infectious disorder, MONDO:0024619), (human disease, MONDO:0700096), (disease, OGMS:0000031)]","[(meningeal tuberculosis, MONDO:0006042)]","[ORDO:641396, DOID:1638, <http://linkedlifedata.com/resource/umls/id/C0275904>, <http://identifiers.org/snomedct/186217006>]",[obo:mondo/patterns/location.yaml]
4,MONDO:0021948,cutaneous tuberculosis,"[(entity, BFO:0000001), (continuant, BFO:0000002), (disposition, BFO:0000016), (realizable entity, BFO:0000017), (specifically dependent continuant, BFO:0000020), (disease, MONDO:0000001), (primary bacterial infectious disease, MONDO:0000314), (extrapulmonary tuberculosis, MONDO:0000368), (integumentary system disorder, MONDO:0002051), (skin disorder, MONDO:0005093), (bacterial infectious disease, MONDO:0005113), (infectious disease, MONDO:0005550), (tuberculosis, MONDO:0018076), (mycobacterial infectious disease, MONDO:0020590), (skin disorder caused by infection, MONDO:0024294), (skin disease caused by bacterial infection, MONDO:0024295), (human disease, MONDO:0700096), (disease, OGMS:0000031)]","[(primary cutaneous

[Back](#overview)

---
<a id='cross_species_analog'></a>
### Get cross-species analog

In [27]:
# Get values for cross-species analog
cross_species_analog = '<https://w3id.org/semapv/vocab/crossSpeciesExactMatch>'

class_df['cross_species_analog'] = class_df['curie'].apply(lambda x: _get_entity_metadata_map_value(x, cross_species_analog))
display(class_df)

,curie,label,parents,children,mappings,conforms_to,cross_species_analog
0,MONDO:0000369,abdominal tuberculosis,"[(entity, BFO:0000001), (continuant, BFO:0000002), (disposition, BFO:0000016), (realizable entity, BFO:0000017), (specifically dependent continuant, BFO:0000020), (disease, MONDO:0000001), (primary bacterial infectious disease, MONDO:0000314), (extrapulmonary tuberculosis, MONDO:0000368), (bacterial infectious disease, MONDO:0005113), (infectious disease, MONDO:0005550), (tuberculosis, MONDO:0018076), (mycobacterial infectious disease, MONDO:0020590), (human disease, MONDO:0700096), (disease, OGMS:0000031)]","[(hepatic tuberculosis, MONDO:0005787), (renal tuberculosis, MONDO:0005938), (splenic tuberculosis, MONDO:0005967), (tuberculous ascites, MONDO:0023153), (tuberculous peritonitis, MONDO:0006000)]","[DOID:0050599, <http://linkedlifedata.com/resource/umls/id/C0740652>, <http://identifiers.org/snomedct/447330002>]",[obo:mondo/patterns/location.yaml],None
1,MONDO:0100481,active tuberculosis,"[(entity, BFO:0000001), (continuant, BFO:0000002), (disposition, BFO:0000016), (realizable entity, BFO:0000017), (specifically dependent continuant, BFO:0000020), (disease, MONDO:0000001), (primary bacterial infectious disease, MONDO:0000314), (bacterial infectious disease, MONDO:0005113), (infectious disease, MONDO:0005550), (tuberculosis, MONDO:0018076), (mycobacterial infectious disease, MONDO:0020590), (human disease, MONDO:0700096), (disease, OGMS:0000031)]",[],[],None,None
2,MONDO:0000813,cardiac tuberculosis,"[(entity, BFO:0000001), (continuant, BFO:0000002), (disposition, BFO:0000016), (realizable entity, BFO:0000017), (specifically dependent continuant, BFO:0000020), (disease, MONDO:0000001), (primary bacterial infectious disease, MONDO:0000314), (extrapulmonary tuberculosis, MONDO:0000368), (cardiovascular disorder, MONDO:0004995), (bacterial infectious disease, MONDO:0005113), (heart disorder, MONDO:0005267), (infectious disease, MONDO:0005550), (tuberculosis, MONDO:0018076), (mycobacterial infectious disease, MONDO:0020590), (human disease, MONDO:0700096), (disease, OGMS:0000031)]",[],"[DOID:0060570, <http://linkedlifedata.com/resource/umls/id/C0041308>, <http://identifiers.org/mesh/D014381>]",[obo:mondo/patterns/location.yaml],None
3,MONDO:0005696,central nervous system tuberculosis,"[(entity, BFO:0000001), (continuant, BFO:0000002), (disposition, BFO:0000016), (realizable entity, BFO:0000017), (specifically dependent continuant, BFO:0000020), (disease, MONDO:0000001), (primary bacterial infectious disease, MONDO:0000314), (extrapulmonary tuberculosis, MONDO:0000368), (central nervous system disorder, MONDO:0002602), (nervous system disorder, MONDO:0005071), (bacterial infectious disease, MONDO:0005113), (infectious disease, MONDO:0005550), (tuberculosis, MONDO:0018076), (infectious disorder of the nervous system, MONDO:0020010), (mycobacterial infectious disease, MONDO:0020590), (central nervous system infectious disorder, MONDO:0024619), (human disease, MONDO:0700096), (disease, OGMS:0000031)]","[(meningeal tuberculosis, MONDO:0006042)]","[ORDO:641396, DOID:1638, <http://linkedlifedata.com/resource/umls/id/C0275904>, <http://identifiers.org/snomedct/186217006>]",[obo:mondo/patterns/location.yaml],None
4,MONDO:0021948,cutaneous tuberculosis,"[(entity, BFO:0000001), (continuant, BFO:0000002), (disposition, BFO:0000016), (realizable entity, BFO:0000017), (specifically dependent continuant, BFO:0000020), (disease, MONDO:0000001), (primary bacterial infectious disease, MONDO:0000314), (extrapulmonary tuberculosis, MONDO:0000368), (integumentary system disorder, MONDO:0002051), (skin disorder, MONDO:0005093), (bacterial infectious disease, MONDO:0005113), (infectious disease, MONDO:0005550), (tuberculosis, MONDO:0018076), (mycobacterial infectious disease, MONDO:0020590), (skin disorder caused by infection, MONDO:0024294), (skin disease caused by bacterial infection, MONDO:0024295), (human disease, MONDO:0700096), (dise